# Project 2 - WIP

Data:
County Boundary Layer (EPSG: 4269): https://www2.census.gov/geo/tiger/GENZ2023/shp/cb_2023_us_county_500k.zip
Census Tracts Layer (EPSG: 4269): https://www2.census.gov/geo/tiger/GENZ2023/shp/cb_2023_us_tract_500k.zip
ACS (API): https://api.census.gov/data/{year}/acs/acs5   {year} is a parameter in the code. Makes code a little more extensible. 2023 is the latest So that is what is used.

Missoula County FIPS: 30063

Total population shown in Table B01003, ref. https://censusreporter.org/tables/B01003/